Initializing staff

In [1]:
import numpy as np
import scipy.integrate as integrate
import lhapdf
import ROOT

Welcome to JupyROOT 6.29/01


Setting up constants

In [2]:
s_h = 27**2 # in GeV (TeV)
p_t = [1,2,3,4]#[0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00] #GeV
i_ = [-4,-3,-2,-1,1,2,3,4]
j_ = [-4,-3,-1,-2,2,1,3,4]
alpha_em=1./137
Nc = 3
C_F = (Nc**2-1)/(2*Nc)
nf = 6
const = 0.04 * pow(10, 7-3) #* np.pi * alpha_em * alpha_em
print(alpha_em, C_F, nf, const)

0.0072992700729927005 1.3333333333333333 6 400.0


Setting up pdf sets

In [3]:
pdfs = [lhapdf.mkPDF("NNPDF30_nlo_as_0118", 0), lhapdf.mkPDF("nNNPDF30_nlo_as_0118_A2_Z1", 0), lhapdf.mkPDF("cteq6l1", 0), #NNPDF23_nlo_as_0118 nNNPDF20_nlo_as_0118_D2
        lhapdf.mkPDF("nCTEQ15HIX_FullNuc_2_1"), lhapdf.mkPDF("nCTEQ15HIX_FullNuc_1_1"), lhapdf.mkPDF("TUJU21_nlo_2_1")]#nNNPDF30_nlo_as_0118_A2_Z1 NNPDF30_nlo_as_0118
#NNSFnu_D_lowQ TUJU21_nlo_1_1

LHAPDF 6.5.4 loading /home/yoren/bnl/lhapdf/LHAPDF-6.5.4/install/share/LHAPDF/NNPDF30_nlo_as_0118/NNPDF30_nlo_as_0118_0000.dat
NNPDF30_nlo_as_0118 PDF set, member #0, version 2; LHAPDF ID = 260000
LHAPDF 6.5.4 loading /home/yoren/bnl/lhapdf/LHAPDF-6.5.4/install/share/LHAPDF/nNNPDF30_nlo_as_0118_A2_Z1/nNNPDF30_nlo_as_0118_A2_Z1_0000.dat
nNNPDF30_nlo_as_0118_A2_Z1 PDF set, member #0, version 1; LHAPDF ID = 30020300
LHAPDF 6.5.4 loading /home/yoren/bnl/lhapdf/LHAPDF-6.5.4/install/share/LHAPDF/cteq6l1/cteq6l1_0000.dat
cteq6l1 PDF set, member #0, version 4; LHAPDF ID = 10042
LHAPDF 6.5.4 loading /home/yoren/bnl/lhapdf/LHAPDF-6.5.4/install/share/LHAPDF/nCTEQ15HIX_FullNuc_2_1/nCTEQ15HIX_FullNuc_2_1_0000.dat
nCTEQ15HIX_FullNuc_2_1 PDF set, member #0, version 2; LHAPDF ID = 110050
LHAPDF 6.5.4 loading /home/yoren/bnl/lhapdf/LHAPDF-6.5.4/install/share/LHAPDF/nCTEQ15HIX_FullNuc_1_1/nCTEQ15HIX_FullNuc_1_1_0000.dat
nCTEQ15HIX_FullNuc_1_1 PDF set, member #0, version 2; LHAPDF ID = 110000
LHAPDF 6.5.

Setting up Histos for various variables

In [4]:
hist = ROOT.TH1D("hist","hist",1000,0,0.1)

# QCD promt photon production
Born cross-sections for $q+g\to q+\gamma$ and $q\bar{q}\to g+\gamma$:
\begin{equation}
\frac{d\sigma^B_{qg}}{dt} = - \frac{1}{N_C} \frac{\alpha_S \alpha_{em}}{s^2} e^2_q \left( \frac{\hat{t}}{\hat{s}} + \frac{\hat{s}}{\hat{t}} \right),~~~~~~~~~~~
\frac{d\sigma^B_{qg}}{dt} = - \frac{1}{N_C} \frac{\alpha_S \alpha_{em}}{s^2} e^2_q \left( \frac{\hat{u}}{\hat{s}} + \frac{\hat{s}}{\hat{u}} \right),~~~~~~~~~~~
\frac{d\sigma^B_{q\bar{q}}}{dt} = - \frac{2C_F}{N_C} \frac{\alpha_S \alpha_{em}}{s^2} e^2_q \left( \frac{\hat{u}}{\hat{t}} + \frac{\hat{t}}{\hat{u}} \right)
\end{equation}
To easily define integration limits we use:
\begin{equation}
\nu=1+\frac{\hat{t}}{\hat{s}}
\end{equation}
where $0<\nu<1$. 
Factorization theorem in Leading Order (LO):
\begin{equation}
E_\gamma \frac{d^3\sigma}{dp^3_\gamma}=\frac{1}{\pi p^4_T} \sum_{ab}{\int^{\nu_{max}}_{\nu_{min}}{d\nu~x_1f_a(x_1,M^2)~x_2f_b(x_2,M^2)~\nu(1-\nu)\hat{s} \frac{d\sigma^{ab}}{d\nu} }}
\end{equation}
where $\nu_{min} \equiv VW \equiv p_T e^y/\sqrt{s}$ and $\nu_{max}\equiv V \equiv 1-p_T e^{-y}/\sqrt{s}$
Factorization theorem in next-to-Leading Order (NLO):
\begin{equation}
E_\gamma \frac{d^3\sigma}{dp^3_\gamma}=\frac{1}{\pi p^4_T} \sum_{ab}{\int^{\nu_{max}}_{\nu_{min}}{d\nu\int^{1}_{\nu_{min}/\nu}{d\omega~x_1f_a(x_1,M^2)~x_2f_b(x_2,M^2)~\nu(1-\nu)~\omega~\hat{s} \frac{d\sigma^{ab}}{d\nu}}}}
\end{equation}
where 
\begin{equation}
\omega \equiv z \equiv \frac{-\hat{u}}{\hat{s}+\hat{t}}
\end{equation}
For LO $\omega = 1$ but for NLO it would be less due to eloss.

Born cross-sections for $q+g\to q+\gamma$ and $q\bar{q}\to g+\gamma$ on $\nu$:
\begin{equation}
\nu(1-\nu)\omega \frac{d^2\sigma^{LO}_{qg}}{d\nu d\omega}=        \frac{1}{N_C}    \frac{\pi \alpha_S \alpha_{em}}{s} e^2_q T_{gq}       \left( 1-\omega \right),~~~~~~~~~~~
\nu(1-\nu)\omega \frac{d^2\sigma^{LO}_{q\bar{q}}}{d\nu d\omega} = \frac{2C_F}{N_C} \frac{\pi \alpha_S \alpha_{em}}{s} e^2_q T_{q\bar{q}} \left( 1-\omega \right)
\end{equation}
Where 
\begin{equation}
T_{gq}=\nu[1+(1-\nu)^2],~~~~~~~~~~~T_{q\bar{q}}=[\nu^2+(1-\nu)^2]
\end{equation}

From https://arxiv.org/abs/hep-ph/9806467v1:
\begin{equation}
\nu(1-\nu)z \frac{d^2\sigma^{LO}_{ij}}{d\nu dz}= \alpha_{em}\alpha_S^2 e_q^2 \left\{ c_3^{ij} \left[\frac{ln(1-z)}{1-z}\right]_+ + c_2^{ij} \left[\frac{1}{1-z}\right]_+ + c_b^{ij} \left[\frac{1}{1-z}\right]_+ \cdot ln\left(\frac{\mu^2}{s} \right)  \right\} \cdot T_{ij}
\end{equation}
Where $ij=qg,q\bar{q}$ and 
\begin{equation}
c_3^{qg}= \frac{1}{N_C}(C_F+2N_C),~~~~~c_2^{qg}=-ln\frac{1-\nu}{\nu}-C_F\left( \frac{3}{4N_C}-\frac{1}{N_C}~ln^{~}\nu \right),~~~~~c_b^{qg}=-\frac{C_F+N_C}{N_C}
\end{equation}
\begin{equation}
c_3^{qq}= \frac{2C_F}{N_C}(4C_F-N_C),~~~~~c_2^{qq}=\frac{C_F n_f}{3N_C}-\frac{11}{6}C_F+2C_F ln(1-\nu)-4\frac{C_F^2}{N_C}ln\frac{1-\nu}{\nu},~~~~~~~c_b^{qq}=-\frac{4C_F^2}{N_C}
\end{equation}

Function for LO Compton and annihilation cross-section

In [5]:
def compton_anih_crossection(nu, y, p, nu_min, nu_max, iset1, iset2, isospin1, isospin2):
  
    x_1 = nu_min/nu
    x_2 = (1-nu_max)/(1-nu)
    
    s = x_1*x_2*s_h

    if 1 - 4*(p)**2/s < 0: return 0
    
    cos_theta = np.sqrt(1 - 4*(p)**2/s)
    u = (-1*s/2)*(1 + cos_theta)
    t = (-1*s/2)*(1 - cos_theta)
    Q2 = p**2#-t
        
    pdf1 = pdfs[iset1]
    pdf2 = pdfs[iset2]
    alpha_s = 0.5*(pdf1.alphasQ2(Q2)+pdf2.alphasQ2(Q2))


    sigma=0.
    for iquark in range(len(i_)):
        i1, i2 = i_[iquark], i_[iquark]
        if isospin1==1: i1 = j_[iquark]
        if isospin2==1: i2 = j_[iquark]
                
        eq2 = (1. + (i_[iquark]+1)%2)**2/9.
        
        pdf_i = max(pdf1.xfxQ2(i1, x_1, Q2),0.)
        pdf_j = max(pdf2.xfxQ2(21, x_2, Q2),0.)
        pdf_k = max(pdf2.xfxQ2(-i2, x_2, Q2),0.)
        pdf_i2 = max(pdf2.xfxQ2(i2, x_2, Q2),0.)
        pdf_j2 = max(pdf1.xfxQ2(21, x_1, Q2),0.)

        sigma_anihhil_dt = (nu**2+(1-nu)**2)*eq2*(2*C_F/Nc)
        #sigma_compton_dt = sigma_anihhil_dt*(-s/u-u/s)/(t/u+u/t)*3/8
        sigma_compton_dt = (1 + (1-nu)**2)*nu*eq2/Nc
            
        sigma += pdf_i*pdf_j*sigma_compton_dt + pdf_i2*pdf_j2*sigma_compton_dt
        sigma += pdf_i*pdf_k*sigma_anihhil_dt
            
    sigma *= alpha_s*alpha_em/p**4
    return  sigma*const

Secondary Function for NLO Compton and annihilation cross-section

In [6]:
def Tij(nu, scat):
    if scat==0:
        return (1 + (1-nu)**2)*nu
    return nu**2+(1-nu)**2

c3=[(C_F+2*Nc)/Nc, 2*C_F/Nc*(4*C_F-Nc)]
cb=[-(C_F+Nc)/Nc, -4*C_F**2/Nc]

def c2(nu, scat):
    if scat==0:
        return -np.log((1-nu)/nu)-C_F*(3./4/Nc-np.log(nu)/Nc)
    return C_F*nf/3/Nc-11./6*C_F+2*C_F*np.log(1-nu)-4*C_F**2/Nc*np.log((1-nu)/nu)


Main Function for NLO Compton and annihilation cross-section

In [7]:
def nlo_compton_anih_crossection(z, nu, y, p, nu_min, nu_max, iset1, iset2, isospin1, isospin2):
  
    x_1 = nu_min/nu/z
    x_2 = (1-nu_max)/(1-nu)
    x_1z = nu_min/nu
    #if x_1 > 1 or x_2 >1 or x_1z>1: return 0
    
    s = x_1*x_2*s_h
    Q2 = p**2#-t
        
    pdf1 = pdfs[iset1]
    pdf2 = pdfs[iset2]
    alpha_s = 0.5*(pdf1.alphasQ2(Q2)+pdf2.alphasQ2(Q2))

    sigma=0.
    TT = [Tij(nu,0)/Nc, Tij(nu,1)*(2*C_F/Nc)]

    pdf_j = max(pdf2.xfxQ2(21, x_2, Q2),0.)
    pdf_j2 = max(pdf1.xfxQ2(21, x_1, Q2),0.)
    pdf_j2_z = max(pdf1.xfxQ2(21, x_1z, Q2),0.)

    corr0 = alpha_s*(c3[0]*np.log(1-z)/(1-z)+(c2(nu,0) + cb[0] * np.log(Q2/s)) /(1-z))
    corr1 = alpha_s*(c3[1]*np.log(1-z)/(1-z)+(c2(nu,1) + cb[1] * np.log(Q2/s)) /(1-z))
    
    for iquark in range(len(i_)):
        i1, i2 = i_[iquark], i_[iquark]
        if isospin1==1: i1 = j_[iquark]
        if isospin2==1: i2 = j_[iquark]
                
        eq2 = (1. + (i_[iquark]+1)%2)**2/9.
        
        pdf_i = max(pdf1.xfxQ2(i1, x_1, Q2),0.)
        pdf_i_z = max(pdf1.xfxQ2(i1, x_1z, Q2),0.)
        pdf_k = max(pdf2.xfxQ2(-i2, x_2, Q2),0.)
        pdf_i2 = max(pdf2.xfxQ2(i2, x_2, Q2),0.)

        sigma_compton_dt = eq2*TT[0]*((pdf_i-pdf_i_z)*pdf_j+pdf_i2*(pdf_j2-pdf_j2_z))*corr0
        sigma_anihhil_dt = eq2*TT[1]*(pdf_i-pdf_i_z)*pdf_k*corr1
        
        sigma += sigma_compton_dt + sigma_anihhil_dt
            
    sigma *= alpha_s*alpha_em/p**4
    return  sigma*const

Integration function

In [8]:
def integration_cs(y, pT, iset1, iset2, isospin1, isospin2):
    
    result=[]
    for p in pT:
        nu_min = p*np.exp(y)/np.sqrt(s_h)
        nu_max = 1-p*np.exp(-y)/np.sqrt(s_h)
        integral = integrate.quad(compton_anih_crossection, nu_min, nu_max, args=(y, p, nu_min, nu_max, iset1, iset2, isospin1, isospin2), epsrel=0.01)
    
        
        result.append(integral)   
                
    return result

$p_T$

In [9]:
pT=[0.5+0.5*i for i in range(10)]
print(pT)

[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]


In [10]:
res_pp=integration_cs(0,pT,4,4,0,0)
res_dd=integration_cs(0,pT,3,3,0,0)
res=[]
cs=0
for i in range(len(res_pp)):
    res.append(res_dd[i][0]/res_pp[i][0])
    cs+=res_pp[i][0]*0.5
print(res_pp)
print(res_dd)
print(res)
print(cs)

[(0.32991470664489253, 0.0003531452985758299), (0.19155945538514763, 0.0005315055464983343), (0.03252318421494012, 0.0002641065042681958), (0.005502764138798929, 2.0866418805505493e-09), (0.0012147797375076923, 5.221533888870745e-08), (0.00031987799807873575, 4.1893239827728457e-07), (9.485887231474041e-05, 2.1795123150304515e-09), (3.0469833841164088e-05, 3.268717701092089e-10), (1.0323657069434657e-05, 9.872281628873084e-10), (3.617552543166866e-06, 6.1384245050259e-10)]
[(0.3102911640500317, 0.0007361618271573139), (0.1758617687947583, 0.0005658718463691648), (0.029316718519580932, 1.6015515210266026e-06), (0.004879239418388509, 5.43039165089125e-08), (0.0010574105458583116, 1.8380630699660165e-08), (0.0002723036748277347, 3.6096818473288405e-08), (7.87073080820106e-05, 1.7557376880229595e-08), (2.4600704190303598e-05, 2.3467761655943283e-11), (8.114380024373858e-06, 7.282874249162622e-10), (2.7749486064504087e-06, 5.443831122981403e-10)]
[0.9405193457593182, 0.9180531884535393, 0.9

Integration Bounds and accuracy

In [11]:
def inner_integral(nu, y, p, nu_min, nu_max, iset1, iset2, isospin1, isospin2):
    
    integral = integrate.quad(nlo_compton_anih_crossection, nu_min/nu, 1.0, args=(nu, y, p, nu_min, nu_max, iset1, iset2, isospin1, isospin2), epsrel=0.01)[0]
    integral += compton_anih_crossection(nu, y, p, nu_min, nu_max, iset1, iset2, isospin1, isospin2)

    return integral

Integration function

In [12]:
def nlo_integration_cs(y, pT, iset1, iset2, isospin1, isospin2):
   
    result=[]
    for p in pT:
        nu_min = p*np.exp(y)/np.sqrt(s_h)
        nu_max = 1-p*np.exp(-y)/np.sqrt(s_h)
        integral = integrate.quad(inner_integral, nu_min, nu_max, args=(y, p, nu_min, nu_max, iset1, iset2, isospin1, isospin2), epsrel=0.05)
        
        result.append(integral)   
                
    return result

In [13]:
res_pp=nlo_integration_cs(0,pT,0,0,0,0)
res_dd=nlo_integration_cs(0,pT,1,1,0,0)
res=[]
cs=0
for i in range(len(res_pp)):
    res.append(res_dd[i][0]/res_pp[i][0])
    cs+=res_pp[i][0]*0.5
print(res_pp)
print(res_dd)
print(res)
print(cs)

/tmp/ipykernel_26318/3974681515.py:3: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral = integrate.quad(nlo_compton_anih_crossection, nu_min/nu, 1.0, args=(nu, y, p, nu_min, nu_max, iset1, iset2, isospin1, isospin2), epsrel=0.01)[0]


[(0.7886670682098978, 0.03043858871098943), (0.198011213013853, 0.0081230011167913), (0.028399325532709498, 0.00026060340600046967), (0.005806427181279009, 0.0001209138927031621), (0.0014436274178345298, 7.093384141509517e-05), (0.00041283735324778063, 6.689953497501715e-06), (0.0001309289859270339, 6.224188116197561e-06), (4.4789713058355154e-05, 1.280238567349982e-06), (1.6150695930894478e-05, 6.072354977424574e-07), (6.018197909622953e-06, 2.550995577839705e-07)]
[(0.6640358762968772, 0.027162520665141437), (0.17810732057351908, 0.005470580104858435), (0.024608455946463527, 0.0007688193049807948), (0.004843577359833356, 0.00016208119519208223), (0.0011616001293515434, 4.860046004729116e-05), (0.0003202488810017236, 8.787229496507392e-06), (9.822352223775529e-05, 1.3126930822997116e-06), (3.28776929572548e-05, 6.094435427849118e-08), (1.1887725401389056e-05, 8.200571057879877e-08), (4.590738986125124e-06, 7.30486952694108e-08)]
[0.841972364592443, 0.8994809832363311, 0.86651550643765

Parallel calculating for LO

In [14]:
import multiprocess as mp

Ntr = 8
pool = mp.Pool(Ntr)

params=[(0,pT,0,0,0,0),(0,pT,0,0,1,0),(0,pT,0,0,0,1),(0,pT,0,0,1,1),(0,pT,1,1,0,0),(0,pT,3,3,0,0),(0,pT,4,4,0,0),(0,pT,5,5,0,0)]

hist_array_lo = pool.starmap(integration_cs, params)
    
pool.close()   

Parallel calculating for NLO

In [15]:
pool = mp.Pool(Ntr)

hist_array_nlo = pool.starmap(nlo_integration_cs, params)
    
pool.close()   

/tmp/ipykernel_26318/3974681515.py:3: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral = integrate.quad(nlo_compton_anih_crossection, nu_min/nu, 1.0, args=(nu, y, p, nu_min, nu_max, iset1, iset2, isospin1, isospin2), epsrel=0.01)[0]
/tmp/ipykernel_26318/3974681515.py:3: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integral = integrate.quad(nlo_compton_anih_crossection, nu_min/nu, 1.0, args=(nu, y, p, nu_min, nu_max, iset1, iset2, isospin1, isospin2), epsrel=0.01)[0]
/tmp/ipykernel_26318/3974681515.py:3: IntegrationWarning: The integral is p

Factors

In [16]:
print("PT:",pT)
nround = 2
res_iso_LO=[]
for ipt in range(len(hist_array_lo[0])):
    res_iso_LO.append(round(0.25*(hist_array_lo[0][ipt][0]+hist_array_lo[1][ipt][0]+hist_array_lo[2][ipt][0]+hist_array_lo[3][ipt][0])/(hist_array_lo[0][ipt][0]+1e-20),nround))
print("isospin  LO:",res_iso_LO)

res_iso_NLO=[]
for ipt in range(len(hist_array_nlo[0])):
    res_iso_NLO.append(round(0.25*(hist_array_nlo[0][ipt][0]+hist_array_nlo[1][ipt][0]+hist_array_nlo[2][ipt][0]+hist_array_nlo[3][ipt][0])/(hist_array_nlo[0][ipt][0]+1e-20),nround))
print("isospin NLO:",res_iso_NLO)

res_nPDF_LO=[]
for ipt in range(len(hist_array_lo[0])):
    res_nPDF_LO.append(round(hist_array_lo[4][ipt][0]/(hist_array_lo[0][ipt][0]+1e-20),nround))
print("nPDF  LO nNNPDF:",res_nPDF_LO)

res_nPDF_LO=[]
for ipt in range(len(hist_array_lo[0])):
    res_nPDF_LO.append(round(hist_array_lo[5][ipt][0]/(hist_array_lo[6][ipt][0]+1e-20),nround))
print("nPDF  LO nCTEQ:",res_nPDF_LO)

res_nPDF_LO=[]
for ipt in range(len(hist_array_lo[0])):
    res_nPDF_LO.append(round(hist_array_lo[4][ipt][0]/(hist_array_lo[6][ipt][0]+1e-20),nround))
print("nPDF  LO TUJU:",res_nPDF_LO)

res_nPDF_NLO=[]
for ipt in range(len(hist_array_nlo[0])):
    res_nPDF_NLO.append(round(hist_array_nlo[4][ipt][0]/(hist_array_nlo[0][ipt][0]+1e-20),nround))
print("nPDF NLO nNNPDF:", res_nPDF_NLO)

res_nPDF_NLO=[]
for ipt in range(len(hist_array_nlo[0])):
    res_nPDF_NLO.append(round(hist_array_nlo[5][ipt][0]/(hist_array_nlo[6][ipt][0]+1e-20),nround))
print("nPDF NLO nCTEQ:", res_nPDF_NLO)

res_nPDF_NLO=[]
for ipt in range(len(hist_array_nlo[0])):
    res_nPDF_NLO.append(round(hist_array_nlo[7][ipt][0]/(hist_array_nlo[6][ipt][0]+1e-20),nround))
print("nPDF NLO TUJU:", res_nPDF_NLO)

res_NLO_LO=[]
for ipt in range(len(hist_array_nlo[0])):
    res_NLO_LO.append(round(hist_array_nlo[4][ipt][0]/(hist_array_lo[0][ipt][0]+1e-20),nround))
print("NLO to LO:",res_NLO_LO)

cs_LO, cs_LO_1, cs_NLO, cs_NLO_1 = 0., 0., 0., 0.
for ipt in range(2, len(pT)):
    cs_LO  += hist_array_lo[0][ipt][0]
    cs_NLO += hist_array_nlo[0][ipt][0]
    cs_LO_1  += hist_array_lo[6][ipt][0]
    cs_NLO_1 += hist_array_nlo[6][ipt][0]
print("cross-section LO and NLO (NNPDF, CTEQ):",round(cs_LO,4), round(cs_NLO,4), round(cs_LO_1,4), round(cs_NLO_1,4))


PT: [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
isospin  LO: [0.96, 0.93, 0.9, 0.88, 0.86, 0.84, 0.82, 0.79, 0.77, 0.75]
isospin NLO: [0.91, 0.93, 0.92, 0.9, 0.88, 0.86, 0.83, 0.81, 0.78, 0.76]
nPDF  LO nNNPDF: [1.05, 0.9, 0.84, 0.81, 0.78, 0.76, 0.74, 0.74, 0.76, 0.8]
nPDF  LO nCTEQ: [0.94, 0.92, 0.9, 0.89, 0.87, 0.85, 0.83, 0.81, 0.79, 0.77]
nPDF  LO TUJU: [2.12, 1.24, 0.89, 0.89, 0.86, 0.83, 0.8, 0.79, 0.81, 0.86]
nPDF NLO nNNPDF: [0.84, 0.9, 0.87, 0.83, 0.8, 0.78, 0.75, 0.73, 0.74, 0.76]
nPDF NLO nCTEQ: [0.93, 0.93, 0.92, 0.9, 0.89, 0.87, 0.85, 0.82, 0.8, 0.78]
nPDF NLO TUJU: [4.94, 1.6, 1.05, 0.99, 0.89, 0.77, 0.66, 0.56, 0.47, 0.41]
NLO to LO: [1.0, 0.68, 0.72, 0.8, 0.87, 0.92, 0.96, 1.0, 1.07, 1.18]
cross-section LO and NLO (NNPDF, CTEQ): 0.042 0.0363 0.0397 0.0343
